In [1]:
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")


CUDA available: True
CUDA version: 12.1
GPU Device: NVIDIA GeForce RTX 4090


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")
wandb_api_key = os.getenv("WANDB_API_KEY")

if not huggingface_api_key:
    raise ValueError("HUGGINGFACE_API_KEY not set")

if not wandb_api_key:
    raise ValueError("WANDB_API_KEY not set")

In [3]:
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer,
    DataCollatorWithPadding
)

from peft import (
    LoraConfig,
    get_peft_model
)

import wandb
import pandas as pd
from datasets import Dataset

In [4]:
from huggingface_hub import login

login(token=huggingface_api_key)

In [5]:
wandb.login(key=wandb_api_key)

run = wandb.init(
    project="repurposed-llm-phishing-classifier-3b-instruct",
    job_type="train",
    anonymous="allow"
)

wandb: Currently logged in as: morganj-lee01 (morganj-lee01-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\morga\_netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [6]:
base_model_path = "../models/llama_models/llama-3.2-3B-Instruct"
new_model_path = "../models/tuned_models/llama-3.2-3B-phishing-classifier-seq-cls-3b-instruct"
train_dataset_path = "../processed_data/train.csv"
test_dataset_path = "../processed_data/test.csv"

In [7]:
train_df = pd.read_csv(train_dataset_path)

train_df.head()

,system,user,assistant
0,You are a classification system designed to ca...,Message for review: write me back please year ...,True
1,You are a classification system designed to ca...,Message for review: I just picked up Razor SDK...,False
2,You are a classification system designed to ca...,"Message for review: vacation goodmorning , i w...",False
3,You are a classification system designed to ca...,"Message for review: On Thu, Aug 08, 2002 at 11...",False
4,You are a classification system designed to ca...,Message for review: wellheads shoreline has se...,False


In [8]:
model_config = {
    "torch_dtype": torch.bfloat16,
    "attn_implementation": "flash_attention_2",
    "device_map": "auto"
}

In [9]:
# WARNING: flash_attention_2 required pip install flash-attn which needs C++ builds tools
# It also takes absolutely forever to compile because it's compiling CUDA kernels

model = AutoModelForSequenceClassification.from_pretrained(
    base_model_path,
    num_labels=2,
    device_map=model_config["device_map"],
    torch_dtype=model_config["torch_dtype"],
    attn_implementation=model_config["attn_implementation"]
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at ../models/llama_models/llama-3.2-3B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)

In [11]:
print("Default special tokens:")
print(tokenizer.special_tokens_map)

print("\nSpecial token IDs:")
for token in tokenizer.all_special_tokens:
    print(f"{token}: {tokenizer.convert_tokens_to_ids(token)}")

Default special tokens:
{'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}

Special token IDs:
<|begin_of_text|>: 128000
<|eot_id|>: 128009


In [12]:
print("Chat template:")
print(tokenizer.chat_template)

Chat template:
{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- if strftime_now is defined %}
        {%- set date_string = strftime_now("%d %b %Y") %}
    {%- else %}
        {%- set date_string = "26 Jul 2024" %}
    {%- endif %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{{- "Cutting Knowledge Date: December 

In [13]:
# COMMENTED OUT BECAUSE INSTRUCT HAS A CHAT TEMPLATE BUT BASE MODEL DOES NOT

# tokenizer.chat_template = None
#
# tokenizer.chat_template = """{% for message in messages %}<|start_header_id|>{{ message['role'] }}<|end_header_id|>{{ message['content'] }}<|eot_id|>{% endfor %}"""


In [14]:
test_messages = [
    {"role": "system", "content": "You are a classification system..."},
    {"role": "user", "content": "Message for review: test"},
    {"role": "assistant", "content": "true"}
]

output = tokenizer.apply_chat_template(test_messages, tokenize=False)
print("Length:", len(output))
print("Content:", output)

Length: 305
Content: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 Jan 2025

You are a classification system...<|eot_id|><|start_header_id|>user<|end_header_id|>

Message for review: test<|eot_id|><|start_header_id|>assistant<|end_header_id|>

true<|eot_id|>


In [15]:
special_tokens = {
    "additional_special_tokens": [
        "<|start_header_id|>",
        "<|end_header_id|>",
        "<|eot_id|>"
    ]
}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(128256, 3072)

In [16]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

In [17]:
from functools import partial

train_dataset = Dataset.from_pandas(pd.read_csv(train_dataset_path))

def format_chat_template_batch(examples, tokenizer):
    formatted_texts = []
    labels = []

    for system, user, assistant in zip(
        examples["system"],
        examples["user"],
        examples["assistant"]
    ):
        row_json = [
            {"role": "system", "content": system},
            {"role": "user", "content": user},
            {"role": "assistant", "content": ""}
        ]
        formatted_texts.append(tokenizer.apply_chat_template(row_json, tokenize=False))
        labels.append(1 if str(assistant).lower() == "true" else 0)

    tokenized = tokenizer(
        formatted_texts,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors=None
    )

    examples.update(tokenized)
    examples["labels"] = labels
    return examples


format_with_tokenizer = partial(format_chat_template_batch, tokenizer=tokenizer)

train_dataset = train_dataset.map(
    format_with_tokenizer,
    batched=True,
    batch_size=100
)

Map:   0%|          | 0/14920 [00:00<?, ? examples/s]

In [18]:
print("Tokenized input:")
input_ids = train_dataset["input_ids"][5]  # This is already a list of token IDs
for token_id in input_ids:
    print(f"{token_id}: {tokenizer.decode([token_id])}")

Tokenized input:
128000: <|begin_of_text|>
128000: <|begin_of_text|>
128006: <|start_header_id|>
9125: system
128007: <|end_header_id|>
271: 


38766: Cut
1303: ting
33025:  Knowledge
2696:  Date
25: :
6790:  December
220:  
2366: 202
18: 3
198: 

15724: Today
2696:  Date
25: :
220:  
2148: 31
4448:  Jan
220:  
2366: 202
20: 5
271: 


2675: You
527:  are
264:  a
24790:  classification
1887:  system
6319:  designed
311:  to
2339:  catch
99197:  phishing
6743:  messages
311:  to
6144:  protect
1274:  people
505:  from
16515:  fraud
12855: sters
323:  and
32638:  criminals
13: .
1472:  You
690:  will
5371:  receive
264:  a
1984:  message
369:  for
3477:  review
323:  and
422:  if
433:  it
374:  is
264:  a
99197:  phishing
2613:  email
11: ,
499:  you
28832:  MUST
6013:  respond
449:  with
1193:  only
364:  '
1904: true
6: '
422:  if
433:  it
374:  is
99197:  phishing
11: ,
477:  or
364:  '
3934: false
6: '
422:  if
433:  it
374:  is
539:  not
99197:  phishing
13: .
25274:  Making
264:  a


In [19]:
test_dataset = Dataset.from_pandas(pd.read_csv(test_dataset_path))

test_dataset = test_dataset.map(
    format_with_tokenizer,
    batched=True,
    batch_size=100
)

Map:   0%|          | 0/3730 [00:00<?, ? examples/s]

In [20]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "down_proj", "up_proj"
    ],
    bias="none",
    task_type="SEQ_CLS",
    inference_mode=False
)

model = get_peft_model(model, peft_config)

In [21]:
training_arguments = TrainingArguments(
    output_dir=new_model_path,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    optim="adamw_torch_fused",
    num_train_epochs=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    do_eval=True,
    warmup_steps=100,
    learning_rate=1e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
    report_to="wandb",
    run_name="actual_run",
    logging_first_step=True,
    logging_dir="../logs",
    logging_strategy="steps",
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=10,
)

In [22]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [23]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
wandb.finish()

In [ ]:
trainer.model.save_pretrained(new_model_path)